LI


# 🏆 Simulação do Campeonato Brasileiro

Este projeto simula o Campeonato Brasileiro de Futebol, considerando atributos fictícios dos times como ataque, defesa e confiança. Os resultados das partidas são gerados usando distribuições de Poisson, permitindo uma simulação estatística realista.

O objetivo é simular todos os jogos de um campeonato de pontos corridos (ida e volta), atualizando a tabela conforme os resultados e exibindo a classificação final.


## 📦 Importação de bibliotecas

In [ ]:

import random
from scipy.stats import poisson

print("Preparando o ambiente no Google Colab...")
!pip install scipy -qqq
print("Ambiente pronto!\n")


Preparando o ambiente no Google Colab...
Ambiente pronto!



## 🧱 Definição da Classe Time
Esta classe representa cada time do campeonato e armazena atributos e estatísticas.

In [ ]:

class Time:
    def __init__(self, nome, ataque, defesa, confianca=0):
        self.nome = nome
        self.ataque = ataque
        self.defesa = defesa
        self.confianca = confianca

        self.pontos = 0
        self.vitorias = 0
        self.empates = 0
        self.derrotas = 0
        self.gols_pro = 0
        self.gols_contra = 0
        self.saldo_gols = 0

        self._jogando_em_casa = False

    def __str__(self):
        return self.nome

    @property
    def jogando_em_casa(self):
        return self._jogando_em_casa

    @jogando_em_casa.setter
    def jogando_em_casa(self, valor):
        self._jogando_em_casa = valor


## ⚽ Lista de times do Campeonato

Valores de ataque e defesa foram inventados para fins de simulação.

In [ ]:

times_brasileirao = [
    Time("Flamengo", ataque=92, defesa=88),
    Time("Palmeiras", ataque=90, defesa=90),
    Time("Botafogo", ataque=87, defesa=83),
    Time("Atlético-MG", ataque=85, defesa=80),
    Time("Fluminense", ataque=82, defesa=78),
    Time("São Paulo", ataque=80, defesa=82),
    Time("Cruzeiro", ataque=79, defesa=77),
    Time("Internacional", ataque=78, defesa=76),
    Time("Grêmio", ataque=77, defesa=75),
    Time("Corinthians", ataque=75, defesa=79),
    Time("Vasco", ataque=72, defesa=70),
    Time("Bahia", ataque=76, defesa=74),
    Time("Fortaleza", ataque=74, defesa=73),
    Time("Athletico-PR", ataque=81, defesa=84),
    Time("Red Bull Bragantino", ataque=73, defesa=72),
    Time("Vitória", ataque=68, defesa=65),
    Time("Atlético-GO", ataque=69, defesa=67),
    Time("Cuiabá", ataque=70, defesa=71),
    Time("Juventude", ataque=67, defesa=66),
    Time("Criciúma", ataque=66, defesa=68),
]


## 🔢 Função calcular_lambdas_gols
Calcula as médias esperadas de gols para cada time com base em seus atributos.

In [ ]:

def calcular_lambdas_gols(time_casa, time_fora):
    media_gols_base_por_time = 1.3

    forca_ataque_casa = time_casa.ataque
    forca_defesa_casa = time_casa.defesa
    forca_ataque_fora = time_fora.ataque
    forca_defesa_fora = time_fora.defesa

    bonus_casa_ataque = 15
    bonus_casa_defesa = 10

    forca_ataque_casa += bonus_casa_ataque
    forca_defesa_casa += bonus_casa_defesa

    forca_ataque_casa += time_casa.confianca * 1.5
    forca_defesa_casa += time_casa.confianca * 0.8
    forca_ataque_fora += time_fora.confianca * 1.5
    forca_defesa_fora += time_fora.confianca * 0.8

    lambda_casa = media_gols_base_por_time * (forca_ataque_casa / 100) * ((200 - forca_defesa_fora) / 100)
    lambda_fora = media_gols_base_por_time * (forca_ataque_fora / 100) * ((200 - forca_defesa_casa) / 100)

    lambda_casa = max(0.5, lambda_casa)
    lambda_fora = max(0.5, lambda_fora)

    return lambda_casa, lambda_fora


## 🕹️ Função simular_jogo
Usa Poisson para sortear a quantidade de gols de cada time na partida.

In [ ]:

def simular_jogo(time_casa, time_fora):
    time_casa.jogando_em_casa = True
    time_fora.jogando_em_casa = False

    lambda_casa, lambda_fora = calcular_lambdas_gols(time_casa, time_fora)

    gols_casa = poisson.rvs(mu=lambda_casa)
    gols_fora = poisson.rvs(mu=lambda_fora)

    time_casa.jogando_em_casa = False
    time_fora.jogando_em_casa = False

    return gols_casa, gols_fora


## 📈 Função atualizar_tabela
Atualiza as estatísticas e confiança dos times com base no resultado.

In [ ]:

def atualizar_tabela(time_casa, time_fora, gols_casa, gols_fora):
    time_casa.gols_pro += gols_casa
    time_casa.gols_contra += gols_fora
    time_casa.saldo_gols = time_casa.gols_pro - time_casa.gols_contra

    time_fora.gols_pro += gols_fora
    time_fora.gols_contra += gols_casa
    time_fora.saldo_gols = time_fora.gols_pro - time_fora.gols_contra

    if gols_casa > gols_fora:
        time_casa.vitorias += 1
        time_casa.pontos += 3
        time_fora.derrotas += 1
        time_casa.confianca = min(10, time_casa.confianca + 1)
        time_fora.confianca = max(-10, time_fora.confianca - 1)
    elif gols_fora > gols_casa:
        time_fora.vitorias += 1
        time_fora.pontos += 3
        time_casa.derrotas += 1
        time_fora.confianca = min(10, time_fora.confianca + 1)
        time_casa.confianca = max(-10, time_casa.confianca - 1)
    else:
        time_casa.empates += 1
        time_fora.empates += 1
        time_casa.pontos += 1
        time_fora.pontos += 1


## 🧮 Função exibir_tabela
Imprime a classificação final com base nas regras de desempate.

In [ ]:

def exibir_tabela(times):
    print("\n" + "="*40 + " TABELA DE CLASSIFICAÇÃO FINAL " + "="*40)
    print(f"{'Pos':<3} {'Time':<20} {'P':<4} {'V':<4} {'E':<4} {'D':<4} {'GP':<4} {'GC':<4} {'SG':<4} {'Conf':<5}")
    print("-" * 100)
    times_ordenados = sorted(times, key=lambda t: (t.pontos, t.saldo_gols, t.gols_pro), reverse=True)

    for i, time in enumerate(times_ordenados):
        print(f"{i+1:<3} {time.nome:<20} {time.pontos:<4} {time.vitorias:<4} {time.empates:<4} {time.derrotas:<4} {time.gols_pro:<4} {time.gols_contra:<4} {time.saldo_gols:<4} {time.confianca:<5.1f}")
    print("=" * 100)
    print("\nSIMULAÇÃO DO CAMPEONATO FINALIZADA!\n")


## 🚀 Função simular_campeonato
Executa toda a lógica da competição com ida e volta entre os times.

In [ ]:

def simular_campeonato(times):
    print("--- INICIANDO SIMULAÇÃO DO CAMPEONATO BRASILEIRO ---")
    jogos = []
    num_times = len(times)
    for i in range(num_times):
        for j in range(i + 1, num_times):
            time1 = times[i]
            time2 = times[j]
            jogos.append((time1, time2))
            jogos.append((time2, time1))
    random.shuffle(jogos)

    total_jogos = len(jogos)
    print(f"\nTotal de jogos a simular: {total_jogos}\n")

    for i, (time_casa, time_fora) in enumerate(jogos):
        if (i + 1) % 50 == 0 or i == 0 or i == total_jogos - 1:
            print(f"--- Jogo {i+1}/{total_jogos}: {time_casa.nome} (C) vs. {time_fora.nome} (F) ---")
        gols_casa, gols_fora = simular_jogo(time_casa, time_fora)
        atualizar_tabela(time_casa, time_fora, gols_casa, gols_fora)
        if (i + 1) % 50 == 0 or i == 0 or i == total_jogos - 1:
            print(f"   Placar: {gols_casa} x {gols_fora}\n")
    exibir_tabela(times)


## ▶️ Iniciar simulação

In [ ]:
simular_campeonato(times_brasileirao)

--- INICIANDO SIMULAÇÃO DO CAMPEONATO BRASILEIRO ---

Total de jogos a simular: 380

--- Jogo 1/380: São Paulo (C) vs. Flamengo (F) ---
   Placar: 3 x 4

--- Jogo 50/380: Red Bull Bragantino (C) vs. São Paulo (F) ---
   Placar: 0 x 1

--- Jogo 100/380: Fortaleza (C) vs. Fluminense (F) ---
   Placar: 0 x 1

--- Jogo 150/380: Flamengo (C) vs. Internacional (F) ---
   Placar: 1 x 2

--- Jogo 200/380: Corinthians (C) vs. Bahia (F) ---
   Placar: 0 x 2

--- Jogo 250/380: Grêmio (C) vs. São Paulo (F) ---
   Placar: 1 x 0

--- Jogo 300/380: Fluminense (C) vs. Flamengo (F) ---
   Placar: 0 x 0

--- Jogo 350/380: Flamengo (C) vs. Grêmio (F) ---
   Placar: 0 x 0

--- Jogo 380/380: Vitória (C) vs. Vasco (F) ---
   Placar: 1 x 1


======================================== TABELA DE CLASSIFICAÇÃO FINAL ========================================
Pos Time                 P    V    E    D    GP   GC   SG   Conf 
---------------------------------------------------------------------------------------------